In [ ]:
# Dependencies
from splinter import Browser
from requests import get
import re
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import select
import urllib.request
from bs4 import BeautifulSoup
import requests
import pymongo

In [ ]:
executable_path = {"executable_path": "chromedriver.exe"}
url = 'https://twitchtracker.com/channels/viewership'

In [ ]:
driver = webdriver.Chrome()
driver.get(url)

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
# print(soup)

In [5]:
pd.read_html(html)[0]
df_tracker_table = pd.read_html(html)[0]
df_tracker_table.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,AVG Viewers,Time Streamed,All Time Peak Viewers,Hours Watched,Rank,Followers Gained,Total Followers,Total Views
0,#1,NaN,tommyinnit,256800,23.5hours,650237,6.03M,1,+796K,3.42M,31.2M
1,#2,NaN,auronplay,166080,86.4hours,313347,14.4M,2,+760K,7.21M,101M
2,#3,NaN,QuackityHQ,123193,29.1hours,232299,3.58M,5,+494K,2.26M,13.9M
3,#4,NaN,karljacobs,110462,23.6hours,206847,2.61M,9,+354K,1.50M,9.51M
4,#5,NaN,GeorgeNotFound,96274,10.6hours,140629,1.02M,12,+235K,2.25M,8.45M


In [6]:
df_tracker_table = df_tracker_table[['Unnamed: 2', 
                                     'AVG Viewers', 
                                     'Time Streamed', 
                                     'All Time Peak Viewers',
                                     'Hours Watched', 
                                     'Rank', 
                                     'Followers Gained', 
                                     'Total Followers', 
                                     'Total Views']]
df_tracker_table

,Unnamed: 2,AVG Viewers,Time Streamed,All Time Peak Viewers,Hours Watched,Rank,Followers Gained,Total Followers,Total Views
0,tommyinnit,256800,23.5hours,650237,6.03M,1,+796K,3.42M,31.2M
1,auronplay,166080,86.4hours,313347,14.4M,2,+760K,7.21M,101M
2,QuackityHQ,123193,29.1hours,232299,3.58M,5,+494K,2.26M,13.9M
3,karljacobs,110462,23.6hours,206847,2.61M,9,+354K,1.50M,9.51M
4,GeorgeNotFound,96274,10.6hours,140629,1.02M,12,+235K,2.25M,8.45M
5,Rubius,89529,84.7hours,344676,7.58M,7,+485K,8.06M,136M
6,BLASTPremier,88628,55.3hours,208744,4.90M,10,+115K,788K,61.8M
7,TheGrefg,77942,169.2hours,2470347,13.2M,4,+1.09M,6.58M,108M
8,ibai,77897,113.8hours,533239,8.87M,8,+554K,4.97M,141M
9,xQcOW,75203,325.5hours,222720,24.5M,3,+280K,7.53M,267M


In [7]:
df_tracker_table_cleaner = df_tracker_table.drop([10], axis='index')
df_tracker_table_cleaner

,Unnamed: 2,AVG Viewers,Time Streamed,All Time Peak Viewers,Hours Watched,Rank,Followers Gained,Total Followers,Total Views
0,tommyinnit,256800,23.5hours,650237,6.03M,1,+796K,3.42M,31.2M
1,auronplay,166080,86.4hours,313347,14.4M,2,+760K,7.21M,101M
2,QuackityHQ,123193,29.1hours,232299,3.58M,5,+494K,2.26M,13.9M
3,karljacobs,110462,23.6hours,206847,2.61M,9,+354K,1.50M,9.51M
4,GeorgeNotFound,96274,10.6hours,140629,1.02M,12,+235K,2.25M,8.45M
5,Rubius,89529,84.7hours,344676,7.58M,7,+485K,8.06M,136M
6,BLASTPremier,88628,55.3hours,208744,4.90M,10,+115K,788K,61.8M
7,TheGrefg,77942,169.2hours,2470347,13.2M,4,+1.09M,6.58M,108M
8,ibai,77897,113.8hours,533239,8.87M,8,+554K,4.97M,141M
9,xQcOW,75203,325.5hours,222720,24.5M,3,+280K,7.53M,267M


In [8]:
df_tracker_table_cleaner.rename(columns= {
                            'Unnamed: 2' : 'streamer_handle',
                            'AVG Viewers' : 'avg_viewers',
                            'Time Streamed' : 'time_streamed(hr)',
                            'All Time Peak Viewers' : 'peak_viewers',
                            'Hours Watched' : 'hours_watched(mil)',
                            'Rank' : "rank",
                            'Followers Gained' : 'followers_gained(thou)', 
                            'Total Followers' : 'total_followers(mil)',
                            'Total Views' : 'total_views(mil)'
                            
}, inplace=True)
df_tracker_table_cleaner.head()

,streamer_handle,avg_viewers,time_streamed(hr),peak_viewers,hours_watched(mil),rank,followers_gained(thou),total_followers(mil),total_views(mil)
0,tommyinnit,256800,23.5hours,650237,6.03M,1,+796K,3.42M,31.2M
1,auronplay,166080,86.4hours,313347,14.4M,2,+760K,7.21M,101M
2,QuackityHQ,123193,29.1hours,232299,3.58M,5,+494K,2.26M,13.9M
3,karljacobs,110462,23.6hours,206847,2.61M,9,+354K,1.50M,9.51M
4,GeorgeNotFound,96274,10.6hours,140629,1.02M,12,+235K,2.25M,8.45M


In [10]:
# Clean up columns
df_tracker_table_cleaner['time_streamed(hr)'] = df_tracker_table_cleaner['time_streamed(hr)'].str.replace(r'hours', '')
df_tracker_table_cleaner['followers_gained(thou)'] = df_tracker_table_cleaner['followers_gained(thou)'].str.replace(r'+', '')
df_tracker_table_cleaner['followers_gained(thou)'] = df_tracker_table_cleaner['followers_gained(thou)'].str.replace(r'K', '')
df_tracker_table_cleaner['followers_gained(thou)'] = df_tracker_table_cleaner['followers_gained(thou)'].str.replace(r'M', '')
df_tracker_table_cleaner['total_followers(mil)'] = df_tracker_table_cleaner['total_followers(mil)'].str.replace(r'M', '')
df_tracker_table_cleaner['total_followers(mil)'] = df_tracker_table_cleaner['total_followers(mil)'].str.replace(r'K', '')
df_tracker_table_cleaner['total_views(mil)'] = df_tracker_table_cleaner['total_views(mil)'].str.replace(r'M', '')
df_tracker_table_cleaner['hours_watched(mil)'] = df_tracker_table_cleaner['hours_watched(mil)'].str.replace(r'M', '')

df_tracker_table_cleaner

,streamer_handle,avg_viewers,time_streamed(hr),peak_viewers,hours_watched(mil),rank,followers_gained(thou),total_followers(mil),total_views(mil)
0,tommyinnit,256800,23.5,650237,6.03,1,796,3.42,31.2
1,auronplay,166080,86.4,313347,14.4,2,760,7.21,101
2,QuackityHQ,123193,29.1,232299,3.58,5,494,2.26,13.9
3,karljacobs,110462,23.6,206847,2.61,9,354,1.50,9.51
4,GeorgeNotFound,96274,10.6,140629,1.02,12,235,2.25,8.45
5,Rubius,89529,84.7,344676,7.58,7,485,8.06,136
6,BLASTPremier,88628,55.3,208744,4.90,10,115,788,61.8
7,TheGrefg,77942,169.2,2470347,13.2,4,1.09,6.58,108
8,ibai,77897,113.8,533239,8.87,8,554,4.97,141
9,xQcOW,75203,325.5,222720,24.5,3,280,7.53,267


In [18]:
# df_tracker_table_cleaner.dtypes
df_tracker_table_cleaner['streamer_handle'] = df_tracker_table_cleaner['streamer_handle'].astype(str)
df_tracker_table_cleaner['avg_viewers'] = df_tracker_table_cleaner['avg_viewers'].astype(int)
df_tracker_table_cleaner['time_streamed(hr)'] = df_tracker_table_cleaner['time_streamed(hr)'].astype(float)
df_tracker_table_cleaner['peak_viewers'] = df_tracker_table_cleaner['peak_viewers'].astype(int)
df_tracker_table_cleaner['hours_watched(mil)'] = df_tracker_table_cleaner['hours_watched(mil)'].astype(float)
df_tracker_table_cleaner['rank'] = df_tracker_table_cleaner['rank'].astype(int)
df_tracker_table_cleaner['followers_gained(thou)'] = df_tracker_table_cleaner['followers_gained(thou)'].astype(float)
df_tracker_table_cleaner['total_followers(mil)'] = df_tracker_table_cleaner['total_followers(mil)'].astype(float)
df_tracker_table_cleaner['total_views(mil)'] = df_tracker_table_cleaner['total_views(mil)'].astype(float)
df_tracker_table_cleaner.dtypes

streamer_handle            object
avg_viewers                 int32
time_streamed(hr)         float64
peak_viewers                int32
hours_watched(mil)        float64
rank                        int32
followers_gained(thou)    float64
total_followers(mil)      float64
total_views(mil)          float64
dtype: object

In [ ]:
df_tracker_table_cleaner.to_csv('top_twenty_streamers_in_last_thirty_days.csv')

In [ ]:
# Below is disregarded code....oh well

In [ ]:
# Empty lists to store data
streamer_handle = []
avg_views = []
time_streamed = []
peak_viewers= []
hours_watched = []
rank_value = []
total_views = []

In [ ]:
# Parse through each tag and pull out text info
channel = soup.find_all('a', {'style' : 'color:inherit'})
views = soup.find_all('td', {'class' : 'color-viewers'})
time = soup.find_all('td', {'class' : 'color-streamed'})
peak_views = soup.find_all('td', {'class' : 'color-viewersMax'})
hours = soup.find_all('td', {'class' : 'color-watched'})
rank = soup.find_all('span', {'class' : 'rank-value'})
all_views = soup.find_all('td', {'class' : 'color-views'})

In [ ]:
for streamer in rank:
    rank_value.append(streamer.get_text())
print(rank_value)

In [ ]:
for streamer in channel:
    streamer_handle.append(streamer.get_text())
print(streamer_handle)

In [ ]:
for streamer in all_views:
    total_views.append(streamer.get_text())
print(total_views)

In [ ]:
for streamer in views:
    avg_views.append(streamer.get_text())
print(avg_views)


In [ ]:
for streamer in time:
    time_streamed.append(streamer.get_text())

print(time_streamed)

In [ ]:
for streamer in peak_views:
    peak_viewers.append(streamer.get_text())
print(peak_viewers)

In [ ]:
for streamer in hours: 
    hours_watched.append(streamer.get_text())
print(hours_watched)

In [ ]:
tracker_df = {'streamer_handle' : streamer_handle,
              'average_viewers':avg_views,
              'hrs_streamed': time_streamed,
              'all_time_peak': peak_viewers, 
              'hrs_watched': hours_watched,
              'rank' : rank_value, 
              'total_views_mil' : total_views}
# tracker_df


In [ ]:
# Put the combined lists into a dataframe
complete_tracker_df = pd.DataFrame.from_dict(tracker_df, orient='index')
complete_tracker_df = complete_tracker_df.transpose()
# complete_tracker_df

In [ ]:
# Clean up columns
complete_tracker_df['hrs_streamed'] = complete_tracker_df['hrs_streamed'].str.replace(r'hours', '')
complete_tracker_df['hrs_watched'] = complete_tracker_df['hrs_watched'].str.replace(r'M', '')
complete_tracker_df['total_views_mil'] = complete_tracker_df['total_views_mil'].str.replace(r'M', '')
complete_tracker_df